In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchsummary import summary
from tqdm import tqdm

In [ ]:
class conv_block(nn.Module):
    def __init__(self, in_channels, out_channels, **kwarg):
        super(conv_block, self).__init__()
        self.relu = nn.ReLU()
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, **kwarg)
        self.batchnorm = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        return self.relu(self.batchnorm(self.conv(x)))

In [ ]:
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_1x1pool):
        super(InceptionBlock, self).__init__()

        self.branch1 = conv_block(in_channels=in_channels, out_channels=out_1x1, kernel_size=1)

        self.branch2 = nn.Sequential(
            conv_block(in_channels=in_channels, out_channels=red_3x3, kernel_size=1),
            conv_block(in_channels=red_3x3, out_channels=out_3x3, kernel_size=3, padding=1)
        )

        self.branch3 = nn.Sequential(
            conv_block(in_channels=in_channels, out_channels=red_5x5, kernel_size=1),
            conv_block(in_channels=red_5x5, out_channels=out_5x5, kernel_size=5, padding=2)
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            conv_block(in_channels=in_channels, out_channels=out_1x1pool, kernel_size=5, padding=2)
        )

    def forward(self, x):
        return torch.cat([self.branch1(x),self.branch2(x),self.branch3(x),self.branch4(x)], 1)

In [ ]:
class InceptionAuxPred(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(InceptionAuxPred, self).__init__()
        self.avgpool = nn.AdaptiveAvgPool2d(4)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.7)
        self.conv = conv_block(in_channels=in_channels, out_channels=128, kernel_size=1)
        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.avgpool(x)

        x = self.conv(x)
        x = x.reshape(x.shape[0], -1)

        x = self.relu(self.fc1(x))
        x = self.dropout(x)

        out = self.fc2(x)
        return out

In [ ]:
class InceptionV3(nn.Module):
    def __init__(self, in_channels=3, num_classes=1000):
        super(InceptionV3, self).__init__()

        self.conv1 = conv_block(in_channels=in_channels, out_channels=64, kernel_size=7, stride=2, padding=3)
        self.conv2 = conv_block(in_channels=64, out_channels=192, kernel_size=3, stride=2, padding=1)

        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception3a = InceptionBlock(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = InceptionBlock(256, 128, 128, 192, 32, 96, 64)

        self.inception4a = InceptionBlock(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = InceptionBlock(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = InceptionBlock(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = InceptionBlock(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = InceptionBlock(528, 256, 160, 320, 32, 128, 128) 

        self.inception5a = InceptionBlock(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = InceptionBlock(832, 384, 192, 384, 48, 128, 128)

        self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1)
        self.dropout = nn.Dropout(p=0.4)
        self.fc = nn.Linear(1024, num_classes)

        self.auxpred1 = InceptionAuxPred(in_channels=512, num_classes=num_classes)
        self.auxpred2 = InceptionAuxPred(in_channels=528, num_classes=num_classes)




    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        
        x = self.conv2(x)
        x = self.maxpool(x)

        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool(x)

        x = self.inception4a(x)
        
        auxpred1 = self.auxpred1(x)
    
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        
        auxpred2 = self.auxpred2(x)

        x = self.inception4e(x)


        x = self.inception5a(x)
        x = self.inception5b(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.dropout(x)
        out = self.fc(x)

        return out, auxpred1, auxpred2



In [ ]:
num_epoch = 5
num_classes = 10
lr = 0.001
batch_size = 64

In [ ]:
my_transform = transforms.Compose([
    transforms.Resize(size=224, ),
    transforms.ToTensor()
])

load_train = datasets.CIFAR10('datasets/', transform=my_transform)
load_test = datasets.CIFAR10('datasets/',train=False, transform=my_transform)

train = DataLoader(load_train, batch_size, shuffle=True)
test = DataLoader(load_test, batch_size, shuffle=True)

load_train[0][0]

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = InceptionV3().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr)
criterion = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(num_epoch):
    for batch_idx, (x_batch, y_batch) in enumerate(tqdm(train)):

        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()

        pred, aux1, aux2 = model(x_batch)

        pred_loss = criterion(pred, y_batch)
        aux1_loss = criterion(aux1, y_batch)
        aux2_loss = criterion(aux2, y_batch)

        loss = pred_loss + 0.3 * aux1_loss + 0.3 * aux2_loss

        loss.backward()
        optimizer.step()

        if batch_idx % 20 == 0:
            print('loss: ', loss)